# CSCI 3104 Assignment 6: Problems

## Instructions

> This assignment is to be completed and uploaded to 
moodle as a python3 notebook. 

> Submission deadlines are posted on moodle. 

> The questions  provided  below will ask you to either write code or 
write answers in the form of markdown.

> Markdown syntax guide is here: [click here](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

> Using markdown you can typeset formulae using latex.

> This way you can write nice readable answers with formulae like thus:

>> The algorithm runs in time $\Theta\left(n^{2.1\log_2(\log_2( n \log^*(n)))}\right)$, 
wherein $\log^*(n)$ is the inverse _Ackerman_ function.

__Double click anywhere on this box to find out how your instructor typeset it. Press Shift+Enter to go back.__


In [2]:
import numpy

## Question 1: Dynamic Programmer Jane's Progress

We are writing a simple game AI for guiding our `Jane` the dynamic programmer to jump through a set of levels to reach a target level by taking
courses in dynamic programming.

The levels positions are numbered 1, ... , n. The character starts at level 1 and the goal is to reach level n (where she becomes
a d.p. ninja) and thus aces CSCI 3104.
After taking a course, she can choose to move up by 1, 4, 5 or 11 levels forward at each step. No backward jumps are available.

![Jane_Programmer At Start of Game](images/jane-picture-p1.png "Jane at the Very Start of the Game" )

Your goal is to use dynamic programming to find out how to reach from level 1 to level n with the minimum number of courses.

## 1(A) Write a recurrence.

Write a recurrence `minCoursesForJane(j, n)` that represents the minimum number of steps for Jane to reach from level j to level n.


In [120]:
def minCoursesForJane(j, n):

    moves = [1,4,5,11]
    iteroptions = []
    
    #Base Case 1: level n reached
    if j == n:
        return 0
    #Base case 2: current level has surpassed n
    if j > n:
        return float('inf')   # return infinity as a warning sign

    k = len(moves)
    assert len(moves) == k
    
    for i in range(k):
        q = 1 + minCoursesForJane(j, n-moves[i])   # count and recurse with the new j, which is moves[i] ahead
        iteroptions.append(q)    # add q to the list of possible number of steps
        optimalValue = min(iteroptions)  # get the minimum number of steps
        
    return optimalValue

In [121]:
## Test Code: Do not edit
print(minCoursesForJane(1, 9)) # should be 2
print(minCoursesForJane(1, 13)) # should be 2
print(minCoursesForJane(1, 19)) # should be 4
print(minCoursesForJane(1, 34)) # should be 3
print(minCoursesForJane(1, 43)) # should be 5

2
2
4
3
5


## 1(B) Memoize the Recurrence.

Assume that n is fixed. The memo table $T[0], \ldots, T[n]$ should store the value of `minCoursesForJane(j, n)`. 

In [122]:
def minCoursesForJane_Memoize(n): 
    
    # start array and fill it with n+1 0s
    memo = numpy.zeros(n+1, dtype=int)
    moves = [1,4,5,11]   # remind me of the acceptable moves

    for i in range(1, n+1):  
        # count the steps needed for every option,
        # store options in memo table
        iteroptions = [1 + memo[i - j] for j in moves if (i - j >= 0)]  
        iteroptions.append(10000000)
        memo[i] = min(iteroptions)
    
    return memo[n-1]

In [123]:
## Test Code: Do not edit
print(minCoursesForJane_Memoize(9)) # should be 2
print(minCoursesForJane_Memoize(13)) # should be 2
print(minCoursesForJane_Memoize(19)) # should be 4
print(minCoursesForJane_Memoize(34)) # should be 3
print(minCoursesForJane_Memoize(43)) # should be 5

2
2
4
3
5


## 1(C) Recover the Solution

Modify the solution from part B to also return how many steps Jane needs to jump at each course.  Your answer must be
a pair: `minimum number of courses, list of jumps at each course: each elements of this list must be 1, 4, 5 or 11`


In [124]:
def minCoursesForJane_Solution(n):
    
    moves = [1,4,5,11]   # acceptable jumps
    
    # create memo tables for the options and solutions, initialize to all zeros and -1 for solutions
    memo = numpy.zeros(n+1, dtype=int)  
    solutions = [-1]*(n+1) 
    jumps = []
    
    # count the steps needed for every option,
    # fill in memo and solutions tables to later recover optimal choice
    for i in range(1,n+1):
        opts = [ (1 + memo[i - j], j)  for j in moves if (i - j >= 0)]
        opts.append((1000000000, -1)) 
        memo[i], solutions[i] = min(opts)
        
    # recovery step
    left = n-1
    while left > 1:
        # append the current solution and update the amount left to check
        jumps.append(solutions[left])   
        left = left - solutions[left]    
    
    return memo[n-1], jumps

In [125]:
## Test Code: Do not edit
print(minCoursesForJane_Solution(9)) # should be 2, [4, 4]
print(minCoursesForJane_Solution(13)) # should be 2, [1, 11]
print(minCoursesForJane_Solution(19)) # should be 4, [1, 1, 5, 11]
print(minCoursesForJane_Solution(34)) # should be 3, [11, 11, 11]
print(minCoursesForJane_Solution(43)) # should be 5, [4, 5, 11, 11, 11]

(2, [4, 4])
(2, [1, 11])
(4, [1, 1, 5, 11])
(3, [11, 11, 11])
(5, [4, 5, 11, 11, 11])


## 1(D) Greedy Solution

Suppose Jane tried a greedy strategy that works as follows. 
Initialize number of courses $c = 0$.

   1. While $n \geq 11$,
      1.1 jump $11$ steps forward, and set $n = n - 11$, $ c = c + 1$
   2. While $n \geq 5$, 
      2.1 jump $5$ steps forward and set $n = n - 5$, $ c = c + 1$
   3. While $n \geq 4$, 
      3.1 jump $4$ steps forward and set $n = n - 4$, $c = c + 1$
   4. Finally, while $n > 1$, 
      4.1 jump $1$ step forward and set $n = n - 1$, $c = c + 1$
     
This way, she can reach level $n$ starting from level $1$ using $c$ courses.

Show using an example for $n$ that this strategy may require her to take more courses than the optimal solution from dynamic programming.

## Answer (Expected Length 3 lines) 


__Answer__

Let's consider the case where Jane is trying to reach level n = 15.
We'll apply the greedy algroithm described above where the largest possible value is picked at every step, starting at initial position j = 1:

__Step 1: j = 1 | c = 0__

j = j + 11 = 12

c = 0 + 1 = 1

__Step 2: j = 12 | c = 1__

(since we're only 3 steps away from 15, the only value we can pick is 1)

j = j + 1 = 13

c = 1 + 1 = 2

__Step 3: j = 13 | c = 2__

j = j + 1 = 14

c = 2 + 1 = 3

__Step 3: j = 14 | c = 3__

j = j + 1 = 15

c = 2 + 1 = 4

__Reached j = 14 | c = 4__

This means it took c = 4 courses to reach n=15 with the greedy algorithm.

With the sequence of c=4 steps: [11, 1, 1, 1]

However, with dynamic programming the optimal solution would've yielded:

__Step 1: j = 1 | c = 0__

j = j + 5 = 6

c = 0 + 1 = 1

__Step 2: j = 6 | c = 1__

j = j + 5 = 11

c = 1 + 1 = 2

__Step 3: j = 11 | c = 2__

j = j + 4 = 15

c = 2 + 1 = 3

With the sequence of c=3 steps: [5, 5, 4]

## Question 2: The Defeat of Kilokahn

Unfortunately, life was not as simple as it seemed in problem 1. Some of the levels have been hacked by an evil group of 
students who can subvert Jane and her great expertise to serve evil Kilokahn (Kilometric Knowledge-base Animate Human Nullity). 

Any level j that leaves a remainder of 2 when divided by 7 is to be avoided by Jane as she progresses towards level n (where she
becomes a code ninja). However, Kilokahn will not be at level $n$ even if $n \mod 7 = 2$.


![Jane_Programmer At Start of Game with Kilokahn lurking](jane-picture-p2.png "Jane at the Very Start of the Game with Kilokahn lurking" )


## 2(A) Write a recurrence.

Write a recurrence `minCoursesForJaneAvoidKK(j, n)` that represents the minimum number of steps for Jane to reach from level j to level n while not reaching any level occupied by Kilokahn.


In [69]:
def minCoursesForJaneAvoidKK(j, n):

    moves = [1,4,5,11]
    iteroptions = []
    
    #Base Case 1: level n reached
    if j == n:
        return 0
    #Base case 2: current level has surpassed n
    if j > n:
        return float('inf')   # return infinity as a warning sign
    
    k = len(moves)
    assert len(moves) == k
    
    for i in range(k):
        if (j+moves[i]) % 7 != 2 or j+moves[i] == n:   # move forward if not stepping into a forbidden step, or if the step is n, where KK cannot be found
            q = 1 + minCoursesForJaneAvoidKK(j+moves[i], n)   # count and recurse with the new j, which is moves[i] ahead
            iteroptions.append(q)   # add q to the list of possible number of steps
        if len(iteroptions) > 0:
            optimalValue = min(iteroptions)   # get the minimum number of steps

    return optimalValue

In [70]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK(1, 9)) # should be 2
print(minCoursesForJaneAvoidKK(1, 13)) # should be 2
print(minCoursesForJaneAvoidKK(1, 19)) # should be 4
print(minCoursesForJaneAvoidKK(1, 34)) # should be 5
print(minCoursesForJaneAvoidKK(1, 43)) # should be 5
print(minCoursesForJaneAvoidKK(1, 55)) # should be 6

2
2
4
5
5
6


## 2(B) Memoize the recurrence in 2(A)

In [126]:
def minCoursesForJaneAvoidKK_Memoize(n): # j is assumed to be 1 
    
    # start memo array and fill it with n+1 0s
    memo = numpy.zeros(n+1, dtype=int)
    moves = [1,4,5,11]

    # count the steps needed for every option,
    # store options in memo table
    for i in range(1, n+1):
        for j in moves:
            if (i-j >= 0 and (i+j)% 7 != 2):   # cover base case to avoid KK
                iteroptions = [1 + memo[i - j]]
                iteroptions.append(10000000)
                memo[i] = min(iteroptions)

    return memo[n]

In [127]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK_Memoize(9)) # should be 2
print(minCoursesForJaneAvoidKK_Memoize(13)) # should be 2
print(minCoursesForJaneAvoidKK_Memoize(19)) # should be 4
print(minCoursesForJaneAvoidKK_Memoize(34)) # should be 5
print(minCoursesForJaneAvoidKK_Memoize(43)) # should be 5
print(minCoursesForJaneAvoidKK_Memoize(55)) # should be 6
print(minCoursesForJaneAvoidKK_Memoize(69)) # should be 8
print(minCoursesForJaneAvoidKK_Memoize(812)) # should be 83

2
2
4
5
5
6
8
83


## 2(C) Recover the solution in terms of number of jumps for each course.

In [128]:
def minCoursesForJaneAvoidKK_Solution(n):
    
    moves = [1,4,5,11] # acceptable jumps
    
    # create memo tables for the options and solutions
    memo = numpy.zeros(n+1, dtype=int)   
    solutions = [-1]*(n+1) 
    jumps = []
    
    # count the steps needed for every option,
    # fill in memo and solutions tables to later recover optimal choice
    for i in range(1,n+1):
        opts = [ (1 + memo[i - j], j)  for j in moves if (i - j >= 0 and (i + j)% 7 != 2)]  # cover base case to avoid KK
        opts.append((1000000000, -1))
        memo[i], solutions[i] = min(opts)
        
    # recovery step
    left = n-1
    while left > 1:
        # append the current solution and update the amount left to check
        jumps.append(solutions[left])
        left = left - solutions[left] 

    return memo[n-1], jumps

In [129]:
## Test Code: Do not edit
print(minCoursesForJaneAvoidKK_Solution(9)) # should be 2, [4, 4]
print(minCoursesForJaneAvoidKK_Solution(13)) # should be 2, [11, 1]
print(minCoursesForJaneAvoidKK_Solution(19)) # should be 4, [4, 5, 4, 5]
print(minCoursesForJaneAvoidKK_Solution(34)) # should be 5, [5, 1, 11, 11, 5]
print(minCoursesForJaneAvoidKK_Solution(43)) # should be 5, [4, 5, 11, 11, 11]
print(minCoursesForJaneAvoidKK_Solution(55)) # should be 6, [5, 11, 11, 11, 11, 5]
print(minCoursesForJaneAvoidKK_Solution(69)) # should be 8, [11, 1, 11, 11, 11, 11, 11, 1]
print(minCoursesForJaneAvoidKK_Solution(812)) # should be 83, [5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11]

(2, [4, 4])
(2, [1, 11])
(4, [1, 1, 5, 11])
(5, [1, 11, 5, 5, 11])
(5, [4, 11, 5, 11, 11])
(6, [5, 11, 11, 5, 11, 11])
(8, [1, 11, 11, 11, 11, 1, 11, 11])
(83, [5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11, 11, 11, 5, 11, 11])


## Question 3: Energize Jane with a budget.

Unfortunately, life was not as simple as it seemed in problem 2. Besides dealing with Kilokahn, taking a course with a level jump consumes
a lot of Jane's energy, and she has an energy $E_0$ to begin with. Each time Jane jumps levels, she loses energy as follows:


| Jump   | Energy Consumed |
|--------|-----------------|
|  1     |       1         |
|  4     |       2         |
|  5     |       3         |
| 11     |       7         |


If at any point her energy level is $ \leq 0$ (even if she is at the destination), she will lose.

Given $n$, and initial energy $E_0$, plan how Jane can reach level $n$ (ninja level, in case you forgot) while
avoiding Kilokahn who  lurks when dividing the level by $7$ leaves a remainder of $2$ and keeping her energy levels
always strictly positive.

----

### 3(A): Write a Recurrence

Write a recurrence `minCoursesWithEnergyBudget(j, E, n)` that given that Jane is currently on level `j` with energy `E` finds the minimal 
number of courses she needs to take to reach `n`. Do not forget the base cases.

In [130]:
def minCoursesWithEnergyBudget(j, e, n):

    moves = [1,4,5,11]
    options = []
    
    #Base Case 1: Jane is out of energy! e is not strictly positive, it's 0 or below
    if e < 1:
        return float('inf')
    
    #Base Case 2: level n reached
    if j == n:
  
        return 0
    #Base case 3: current level has surpassed n
    if j > n:

        return float('inf')   # return infinity as a warning sign

    k = len(moves)
    assert len(moves) == k
    
    for i in range(k):
        
        if (j+moves[i]) % 7 != 2 or j+moves[i] == n:   # move forward if not stepping into a forbidden step, or if the step is n, where KK cannot be found   
            
            if moves[i] == 1:
                q = 1 + minCoursesWithEnergyBudget(j+moves[i], e-1, n)   # count and recurse with the new j, which is moves[i] ahead, reduce energy by 1
                options.append(q)   # add q to the list of possible number of steps
    
            if moves[i] == 4:
                q = 1 + minCoursesWithEnergyBudget(j+moves[i], e-2, n)   # count and recurse with the new j, which is moves[i] ahead, reduce energy by 2
                options.append(q)   # add q to the list of possible number of steps
                
            if moves[i] == 5:
                q = 1 + minCoursesWithEnergyBudget(j+moves[i], e-3, n)   # count and recurse with the new j, which is moves[i] ahead, reduce energy by 3
                options.append(q)   # add q to the list of possible number of steps
                
            if moves[i] == 11:
                q = 1 + minCoursesWithEnergyBudget(j+moves[i], e-7, n)   # count and recurse with the new j, which is moves[i] ahead, reduce energy by 7
                options.append(q)   # add q to the list of possible number of steps
            
        if len(options) > 0:
            bestValue = min(options)   # get the minimum number of steps
            
    return bestValue

In [287]:
# test code do not edit
print(minCoursesWithEnergyBudget(1, 25, 10)) # must be 2
print(minCoursesWithEnergyBudget(1, 25, 6)) # must be 1
print(minCoursesWithEnergyBudget(1, 25, 30)) # must be 5
print(minCoursesWithEnergyBudget(1, 16, 30)) # must be 7
print(minCoursesWithEnergyBudget(1, 18, 31)) # must be 7
print(minCoursesWithEnergyBudget(1, 22, 38)) # must be 7
print(minCoursesWithEnergyBudget(1, 32, 55)) # must be 11
print(minCoursesWithEnergyBudget(1, 35, 60)) # must be 12

2
1
5
7
7
7
11
12


## 3(B): Memoize the Recurrence

Write a memo table to memoize the recurrence. Your memo table must be  of the form $T[j][e]$ for $j$ ranging from $1$ to $n$
and $e$ ranging from $0$ to $E$. You will have to handle the base cases carefully.

In [131]:
def minCoursesWithEnergyBudget_Memoize(E, n):
    
    # create memo table with appropriate number of rows and columns
    memo = numpy.zeros((E+1, n+1), dtype=int)  
    
    # remind me of moves allowed and corresponding energy costs
    moves = [1, 4, 5, 11]
    energy = [1, 2, 3, 7]
    
    for e in range(1, E+1):
        
        for m in range(1, n+1):
            options = []  # array to store options
            
            for i in range(len(moves)):
                if (m - moves[i] >= 0) and (e - energy[i] > 0):  # cover base cases: don't drop below n, keep energy positive...
                    if (m - moves[i] + 1) %7 != 2:  # and do NOT step into KK's realm
                        options.append(1 + memo[e - energy[i]][m - moves[i]])
                    options.append(1000000)
                options.append(1000000)
                
            memo[e][m] = min(options)  # keep the most optimal option
            
    return memo[E][n-1]

In [132]:
# test code do not edit
print(minCoursesWithEnergyBudget_Memoize(25, 10)) # must be 2
print(minCoursesWithEnergyBudget_Memoize(25, 6)) # must be 1
print(minCoursesWithEnergyBudget_Memoize(25, 30)) # must be 5
print(minCoursesWithEnergyBudget_Memoize(16, 30)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(18, 31)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(22, 38)) # must be 7
print(minCoursesWithEnergyBudget_Memoize(32, 55)) # must be 11
print(minCoursesWithEnergyBudget_Memoize(35, 60)) # must be 12

2
1
5
7
7
7
11
12


## 3(C): Recover the Solution

Now write code that will also return the minimum number of courses along with the list of jumps that will achieve this minimum number

In [48]:
def minCoursesWithEnergyBudget_Solution(E, n):

    # remind me of moves allowed and corresponding energy costs
    moves = [1, 4, 5, 11]
    energy = [1, 2, 3, 7]
    
    # initialize 2D memo table to all zeros
    memo = numpy.zeros((E+1, n+1), dtype=int)
    
    # initialize arrays to store solutions and keep track of energy/jumps
    solutions = numpy.zeros((E+1, n+1), dtype=tuple)
    ecount = numpy.zeros((E+1, n+1), dtype=int)
    jcount = numpy.zeros((E+1, n+1), dtype=int)
    
    for e in range(1, E+1):
        for m in range(1, n+1):
            memo_values = []
            ecount_values = []
            jcount_values = []
            
            # count the steps needed for every option,
            # fill in memo, count and solutions tables to later recover optimal choice
            for i in range(len(moves)):
                if (m-moves[i] >= 0) and (e-energy[i] > 0):  # cover base cases: keep energy positive and do not visit KK
                    if ((m-moves[i] + 1) %7 != 2):
                        
                        # keep the step and energy count, fill other rows with undesired values
                        memo_values.append(1 + memo[e-energy[i]][m-moves[i]])
                        ecount_values.append(e-energy[i])
                        jcount_values.append(m)
                        
                    memo_values.append(1000000)
                    ecount_values.append(-1000000)
                    jcount_values.append(1000000)
                
                memo_values.append(1000000)
                ecount_values.append(-1000000)
                jcount_values.append(1000000)
            
            # store min steps, max energy
            memo[e][m] = min(memo_values)
            ecount[e][m] = max(ecount_values)
            jcount[e][m] = min(jcount_values)
        
    # recovery step
    jleft = n-1
    eleft = E
    totaljumps = []

    # append the current solution and update the amount left to check
    while (jleft > 0):
        totaljumps.append(jcount[eleft][jleft])
        eleft = eleft - ecount[eleft][jleft]
        jleft = jleft - jcount[eleft][jleft]
        
    return memo[E][n-1], totaljumps

In [1]:
# test code do not edit
print(minCoursesWithEnergyBudget_Solution(25, 10)) # must be 2, [4,5]
print(minCoursesWithEnergyBudget_Solution(25, 6)) # must be 1, [5]
print(minCoursesWithEnergyBudget_Solution(25, 30)) # must be 5, [4, 5, 4, 5, 11]
print(minCoursesWithEnergyBudget_Solution(16, 30)) # must be 7, [4, 5, 4, 4, 4, 4, 4]
print(minCoursesWithEnergyBudget_Solution(18, 31)) # must be 7, [4, 5, 4, 4, 4, 4, 5]
print(minCoursesWithEnergyBudget_Solution(22, 38)) # must be 7,  [4, 5, 4, 4, 4, 5, 11]
print(minCoursesWithEnergyBudget_Solution(32, 55)) # must be 11, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5]
print(minCoursesWithEnergyBudget_Solution(35, 60)) # must be 12, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5, 5]

(2, [4, 5])
(1, [5])
(5, [4, 5, 4, 5, 11])
(7, [4, 5, 4, 4, 4, 4, 4])
(7, [4, 5, 4, 4, 4, 4, 5])
(7, [4, 5, 4, 4, 4, 5, 11])
(11, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5])
(12, [4, 5, 4, 4, 4, 4, 5, 4, 4, 11, 5, 5])


----

## Question 4: Subset Sum Problem

We are given a set of whole numbers $S:\ \{ n_1, \ldots, n_k \}$ and a number $N$.
Our goal is to choose a subset of numbers $T:\ \{ n_{i_1}, \ldots, n_{i_j} \} \subseteq S$ such that

   (a) $\sum_{l=1}^j n_{i_l}  \leq N$, the sum of chosen numbers is less than or equal to $N$, 

   (b) The difference $N - \sum_{l=1}^j n_{i_l} $ is made as small as possible.

 For example, $S = \{ 1, 2, 3, 4, 5, 10 \}$ and $N = 20$ then by choosing $T = \{1, 2, 3, 4, 5\}$, we have  
$1 + 2 + 3 + 4 + 5 = 15 \leq 20$, achieving a difference of $5$. However, if we chose $T = \{ 2,3,5,10\}$ 
we obtain a sum of $2 + 3 + 5 + 10 = 20$ achieving the smallest possible difference of $0$.


Therefore the problem is as follows:

  * Inputs: list  $S: [n_1, \ldots, n_k]$ and number $N$.
  * Output: a list $T$ of elements from $S$ such that sum of elements of $T$ is  $\leq N$ and $N - \sum_{e \in T} e$ is the smallest possible.

The subsequent parts to this problem ask you to derive a dynamic programming solution to this problem.

__Note:__ Because $S$ and $T$ are viewed as sets, each element in the set may occur exactly once.

 ## 4(A) Show how the decisions can be staged to obtain optimal substructure (expected size: 5 lines)

__Answer__

A given problems has Optimal Substructure Property if the optimal solution to the given problem can be obtained by computing the optimal solutions of its subproblems. 
Suppose we are trying to reach N by adding up different numbers from set S. We will start by picking an element j from S, we'll subtract it from N, and solve the problem for N-j. The steps can be summarized as follows:

1. We have an array of numbers $[j_1, \ldots, j_k]$. We first decide on an item j to subtract from N.
2. As a result of the above step, we get a new target value N-j. We can now consider the same problem in #1, but with input N-j.
3. If the previous subproblem has a solution, we join it to the larger problem's solution. We continue to reduce N, picking an element j from the set to subtract from it, until the problem can no longer be reduced.

Thus, we see this problem has optimal substructure.

## 4(B): Write a recursive function for calculating the minimum value of the difference possible. 

In [135]:
def minSubsetDifference_recursive(N, s_list): 
    
    k = len(s_list)
    assert len(s_list) == k
    
    # Base case 1: N reached 0, return 0
    if N == 0:
        return 0
    
    # Base case 2: N is less than 0, return its absolute value
    if N < 0:
        return abs(N)
    
    # Base case 3: Sum of all numbers is less than N, return the difference
    if sum(s_list) < N:
        return N-sum(s_list)

    options = []
    
    # loop and recurse, reducing N at every step to solve smaller subproblems
    # note items in a set cannot be reused, so recursive call uses list without current element
    for i in range(k):
        q = minSubsetDifference_recursive(N - s_list[i], s_list[:i] + s_list[i+1:])
        options.append(q)
    
    # get the minimum difference possible to return it
    min_difference = min(options)
        
    return min_difference

In [136]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference_recursive(15, [1, 2, 3, 4, 5, 10])) # Should be zero
print(minSubsetDifference_recursive(26, [1, 2, 3, 4, 5, 10])) # should be 1
print(minSubsetDifference_recursive(23, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(18, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(9, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_recursive(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1
print(minSubsetDifference_recursive(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0
print(minSubsetDifference_recursive(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1

0
1
0
0
0
1
0
1


## 4(C): Memoize the recurrence above. 

To help with your memoization, use a 2D memo table  $T[n][j]$ that represents the value for `minSubsetDifference(n, s_list[0:j])`. 

In [137]:
def minSubsetDifference_Memoize(N, s_list):

    # initialize N as target to avoid changing its value
    # initialize solutions array and difference var
    target = N
    solution_list = s_list
    difference = 0
    
    for i in range(2**len(s_list)): 
        # use the 1s and 0s to check all the possible sums
        options = str(numpy.base_repr(i, base=2, padding=len(s_list))[-len(s_list):])
        memo = []  # initialize memo table
        
        for j in range(len(options)):
            if options[j]=='1':
                memo += [s_list[j]]
        
        # count 1s for current sum
        num_sum = sum(memo)
        
        # base case -> if the current sum is <= N, and difference is under N, save in solutions list and update target
        if num_sum <= N:
            m = N - num_sum
            if m < target:
                solution_list = memo
                target = m
           
        # get the absolute value of the difference with N
        min_difference = abs(N - sum(solution_list))

    return min_difference

In [138]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference_Memoize(15, [1, 2, 3, 4, 5, 10])) # Should be 0
print(minSubsetDifference_Memoize(26, [1, 2, 3, 4, 5, 10])) # should be 1
print(minSubsetDifference_Memoize(23, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(18, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(9, [1, 2, 3, 4, 5, 10])) # should be 0
print(minSubsetDifference_Memoize(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1
print(minSubsetDifference_Memoize(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0
print(minSubsetDifference_Memoize(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1

0
1
0
0
0
1
0
1


## 4(D): Write code to recover the solution

In [3]:
def minSubsetDifference(N, s_list):
    
    # initialize N as target to avoid changing its value
    # initialize solutions array and difference var
    target = N
    solution_list = s_list
    difference = 0
    
    for i in range(2**len(s_list)):  
        # use the 1s and 0s to check all the possible sums
        options = str(numpy.base_repr(i, base=2, padding=len(s_list))[-len(s_list):])
        memo = []  # initialize memo table
        
        for j in range(len(options)):
            if options[j]=='1':
                memo += [s_list[j]]
        
        # count 1s for current sum
        num_sum = sum(memo)
        
        # base case -> if the current sum is <= N, and difference is under N, save in solutions list and update target
        if num_sum <= N:
            m = N - num_sum
            if m < target:
                solution_list = memo
                target = m
        
        # get the absolute value of the difference with N
        min_difference = abs(N - sum(solution_list))

    return min_difference, solution_list

In [4]:
# Code for testing your solution
# DO NOT EDIT
print(minSubsetDifference(15, [1, 2, 3, 4, 5, 10])) # Should be 0, [5, 10]
print(minSubsetDifference(26, [1, 2, 3, 4, 5, 10])) # should be 1, [1, 2, 3, 4, 5, 10]
print(minSubsetDifference(23, [1, 2, 3, 4, 5, 10])) # should be 0, [1, 3, 4, 5, 10]
print(minSubsetDifference(18, [1, 2, 3, 4, 5, 10])) # should be 0, [3, 5, 10]
print(minSubsetDifference(9, [1, 2, 3, 4, 5, 10])) # should be 0, [4, 5]
print(minSubsetDifference(457, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1, [23, 94, 339]
print(minSubsetDifference(512, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 0, [48, 152, 312]
print(minSubsetDifference(616, [11, 23, 37, 48, 94, 152, 230, 312, 339, 413])) # should be 1, [23, 37, 48, 94, 413]

(0, [5, 10])
(1, [1, 2, 3, 4, 5, 10])
(0, [1, 3, 4, 5, 10])
(0, [3, 5, 10])
(0, [4, 5])
(1, [23, 94, 339])
(0, [48, 152, 312])
(1, [23, 37, 48, 94, 413])


## 4 (E): Greedy Solution

Suppose we use the following greedy solution to solve the problem.
  * $T = \emptyset$
  * While ( $ N \geq 0 $) 
    * Select the largest element $e$ for $S$ that is smaller than $N$
    * Remove $e$ from $S$
    * Add $e$ to $T$
    * N = N - e
  * return (N, T)
  
Using an example, show that the greedy algorithm does not necessarily produce the optimal solution.

### Answer (4 lines)

__Answer__

In the test cases above, we worked with N = 23, S = [1, 2, 3, 4, 5, 10].
As shown, dynamic programming yields the optimal solution for the minimum difference between N and a sum of elements in set S as 0. However, if we decided to use the greedy algorithm described above, we would have obtained a minimum difference of 1. 

At each step, pick largest unchosen element e of S to subtract from N: 

N=23

__Step 1:__

e = 10

N = N - e = 13

__Step 2:__

e = 5

N = N - e = 8

__Step 3:__

e = 4

N = N - e = 4

__Step 4:__

e = 3

N = N - e = 1

We are thus left with 1, and because our next largest element of S is 2, we have to stop there. 
This gives us a difference of 23-10-5-4-3 = 1, rather than the optimal 0 difference that we get by picking [10, 5, 4, 3, 1], since 23-10-5-4-3-1 = 0.

## Testing your solutions -- Do not edit code beyond this point